# NLP Assignment 2 

In [4]:
import pandas as pd
import json



## Loading the dataset

### ATIS

In [1]:
# -------------------------
# Load ATIS (from HuggingFace hosted CSVs)
# -------------------------
splits = {
    'train': 'atis_train.csv',
    'test': 'atis_test.csv'
}

In [ ]:
atis_train = pd.read_csv("hf://datasets/tuetschek/atis/" + splits["train"])
atis_test = pd.read_csv("hf://datasets/tuetschek/atis/" + splits["test"])

print("ATIS Train shape:", atis_train.shape)
print("ATIS Test shape:", atis_test.shape)
print(atis_train.head())

### SLURP

In [2]:
# -------------------------
# Load SLURP (local jsonl files)
# -------------------------
def load_jsonl(path):
    data = []
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    return data


In [5]:

slurp_train = load_jsonl("slurp_1/train.jsonl")
slurp_dev = load_jsonl("slurp_1/devel.jsonl")
slurp_test = load_jsonl("slurp_1/test.jsonl")
slurp_train_synth = load_jsonl("slurp_1/train_synthetic.jsonl")

print("SLURP Train size:", len(slurp_train))
print("Example SLURP entry:", slurp_train[0])

SLURP Train size: 11514
Example SLURP entry: {'slurp_id': 9024, 'sentence': 'event', 'sentence_annotation': 'event', 'intent': 'calendar_set', 'action': 'set', 'tokens': [{'surface': 'event', 'id': 0, 'lemma': 'event', 'pos': 'NN'}], 'scenario': 'calendar', 'recordings': [{'file': 'audio-1501754435.flac', 'wer': 0.0, 'ent_wer': None, 'status': 'correct'}, {'file': 'audio-1501407267-headset.flac', 'wer': 0.0, 'ent_wer': None, 'status': 'correct'}, {'file': 'audio-1501407267.flac', 'wer': 0.0, 'ent_wer': None, 'status': 'correct'}, {'file': 'audio-1501771798-headset.flac', 'wer': 0.0, 'ent_wer': None, 'status': 'correct'}, {'file': 'audio-1501771798.flac', 'wer': 0.0, 'ent_wer': None, 'status': 'correct'}, {'file': 'audio-1490705711-headset.flac', 'wer': 0.0, 'ent_wer': None, 'status': 'correct'}, {'file': 'audio-1490705711.flac', 'wer': 0.0, 'ent_wer': None, 'status': 'correct'}, {'file': 'audio-1494416970-headset.flac', 'wer': 0.0, 'ent_wer': None, 'status': 'correct'}, {'file': 'audio